In [ ]:
# TODO : define the optimizer, the checkpoints, the logs

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

from losses import total_loss
from maptd_model import maptd_model
from pipeline_v2 import get_dataset

In [ ]:
TILE_SIZE = 512
BATCH_SIZE = 16
TRAINING_STEPS = 5000
img_dir = 'D:/Gerasimos/Toponym_Recognition/MapTD_General/MapTD/data/general_dataset/images'
json_dir = 'D:/Gerasimos/Toponym_Recognition/MapTD_General/MapTD/data/general_dataset/json'

In [ ]:
def show_score_map(tile, gt_score_map, pred_score_map, threshold=None):
    pred_score_map = np.where(pred_score_map > threshold, 1, 0)
    COLORMAP = 'gray'
    fig, ax = plt.subplots(1, 3)
    ax[0].imshow(np.squeeze(tf.cast(tile, tf.uint8)))
    ax[1].imshow(np.squeeze(gt_score_map), cmap=COLORMAP)
    ax[2].imshow(np.squeeze(pred_score_map), cmap=COLORMAP)
    plt.show()

In [ ]:
def train_step(model, tile, step):
    with tf.GradientTape() as tape:
        pred_score_map, pred_geo_map = model(tile, training=True)

        loss = total_loss(score_map, pred_score_map, geo_map, 
                              pred_geo_map, training_mask)
        loss_history.append(loss)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [ ]:
ds = get_dataset(img_dir, json_dir, '*', tile_size=TILE_SIZE, batch_size=BATCH_SIZE)

In [ ]:
maptd = maptd_model(input_size=TILE_SIZE)
models_dir = './models'
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_history = []
for step, (tile, score_map, geo_map, training_mask) in \
            ds.repeat().take(TRAINING_STEPS).enumerate():
    
    train_step(maptd, tile, step)
        
    #if step == 0:
        #model_path = os.path.join(models_dir, f'model_at_step_{step.numpy() + 1}')
        #model.save(model_path)

    #if ((step + 1) % 10000) == 0:
        #show_score_map(tile[5], score_map[5], pred_score_map[5], threshold=0.8)
        #model_path = os.path.join(models_dir, f'model_at_step{step.numpy()}')
        #model.save(model_path)

In [ ]:
test_ds = get_dataset(img_dir, json_dir, '*', tile_size=TILE_SIZE, batch_size=1)

In [ ]:
modelpath = 'D:/Gerasimos/Toponym_Recognition/MapTD_General/MapTD_TF2/models/model_at_step109999'

model = tf.keras.models.load_model(modelpath)
models_dir = './models'
TEST_STEPS = 50
for step, (tile, score_map, geo_map, training_mask) in test_ds.repeat().take(TEST_STEPS).enumerate():
    pred_score_map, pred_geo_map = model(tile, training=False)
    show_score_map(tile, score_map, pred_score_map, threshold=0.8)